<a href="https://colab.research.google.com/github/santwan/upi-fraud-transaction-detection/blob/main/Dataset_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.3 MB/s eta 0:00:00


# 1. Imports and Global Config

## Importing all the necessary library for data generationn

In [6]:

import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

## **Initializing Faker**

In [7]:
# Initialize Faker for India
fake = Faker('en_IN')

## **CONFIGURATION PARAMETER**

In [8]:
# --- Configuration Parameters ---

NUM_USERS = 5000
TOTAL_TRANSACTIONS = 150000
FRAUD_RATIO = 0.01  # 1% Fraud


## **BANK LIST IN INDIA**

In [9]:
banks = [
    'HDFC Bank',
    'ICICI Bank',
    'SBI',
    'Axis Bank',
    'Kotak Mahindra Bank',
    'Punjab National Bank',
    'Bank of Baroda',
    'IDFC FIRST Bank',
    'IndusInd Bank',
    'Yes Bank',
    'Union Bank of India',
    'Canara Bank',
    'Bank of India',
    'Federal Bank',
    'Central Bank of India',
    'UCO Bank',
    'Indian Bank',
    'South Indian Bank',
    'Karur Vysya Bank',
    'RBL Bank',
    'Bandhan Bank',
    'AU Small Finance Bank',
    'Jana Small Finance Bank',
    'City Union Bank'
]


## **UPI DOMAIN, DEVICE TYPE, LOCATION, & FOREIGN CITIES**

In [10]:
upi_id_domains = ['@ybl', '@okaxis', '@apl', '@sbi', '@hdfcbank', '@ptsbi']

device_types = ['Android', 'iOS']

merchant_categories = ['Food', 'Grocery', 'Fuel', 'Entertainment', 'Shopping', 'Healthcare', 'Education', 'Transport', 'Utilities' ]


## **indian Cities List with their population**

In [11]:
#  Added a population data to the indian_cities list and use random.choices for a weighted selection.
# This ensures that larger cities are chosen more frequently when generating user profiles.

indian_cities = [
    {"city": "Mumbai", "lat": 19.0760, "lon": 72.8777, "population": 20.4},
    {"city": "Delhi", "lat": 28.7041, "lon": 77.1025, "population": 30.3},
    {"city": "Bengaluru", "lat": 12.9716, "lon": 77.5946, "population": 12.7},
    {"city": "Hyderabad", "lat": 17.3850, "lon": 78.4867, "population": 10.5},
    {"city": "Ahmedabad", "lat": 23.0225, "lon": 72.5714, "population": 8.6},
    {"city": "Chennai", "lat": 13.0827, "lon": 80.2707, "population": 11.5},
    {"city": "Kolkata", "lat": 22.5726, "lon": 88.3639, "population": 15.0},
    {"city": "Pune", "lat": 18.5204, "lon": 73.8567, "population": 6.9},
    {"city": "Jaipur", "lat": 26.9124, "lon": 75.7873, "population": 4.1},
    {"city": "Lucknow", "lat": 26.8467, "lon": 80.9462, "population": 3.6},
    {"city": "Kanpur", "lat": 26.4499, "lon": 80.3319, "population": 3.1},
    {"city": "Nagpur", "lat": 21.1458, "lon": 79.0882, "population": 2.9},
    {"city": "Indore", "lat": 22.7196, "lon": 75.8577, "population": 3.5},
    {"city": "Bhopal", "lat": 23.2599, "lon": 77.4126, "population": 2.5},
    {"city": "Patna", "lat": 25.5941, "lon": 85.1376, "population": 2.4},
    {"city": "Vadodara", "lat": 22.3072, "lon": 73.1812, "population": 2.2},
    {"city": "Ludhiana", "lat": 30.9000, "lon": 75.8573, "population": 1.7},
    {"city": "Agra", "lat": 27.1767, "lon": 78.0081, "population": 1.9},
    {"city": "Varanasi", "lat": 25.3176, "lon": 82.9739, "population": 1.4},
    {"city": "Surat", "lat": 21.1702, "lon": 72.8311, "population": 8.1},
    {"city": "Coimbatore", "lat": 11.0168, "lon": 76.9558, "population": 2.2},
    {"city": "Thane", "lat": 19.2183, "lon": 72.9781, "population": 2.6},
    {"city": "Meerut", "lat": 28.9845, "lon": 77.7064, "population": 1.6},
    {"city": "Rajkot", "lat": 22.3039, "lon": 70.8022, "population": 1.9},
    {"city": "Ranchi", "lat": 23.3441, "lon": 85.3096, "population": 1.5},
    {"city": "Guwahati", "lat": 26.1445, "lon": 91.7362, "population": 1.2},
    {"city": "Chandigarh", "lat": 30.7333, "lon": 76.7794, "population": 1.2},
    {"city": "Mysuru", "lat": 12.2958, "lon": 76.6394, "population": 1.0},
    {"city": "Dehradun", "lat": 30.3165, "lon": 78.0322, "population": 0.8},
    {"city": "Visakhapatnam", "lat": 17.6868, "lon": 83.2185, "population": 2.0}
]

# Create separate lists for cities and their weights
city_names = [c["city"] for c in indian_cities]
city_weights = [c["population"] for c in indian_cities]

## **City to ISP mapping**

In [12]:
CITY_TO_ISP_PREFIXES = {
    "Mumbai": [
        ("49.36",    "Jio"),
        ("103.23",   "Airtel"),
        ("59.89",    "Tata"),
        ("106.51",   "ACT")
    ],
    "Delhi": [
        ("27.97",    "Airtel"),
        ("49.205",   "Jio"),
        ("115.247",  "Hathway"),
        ("117.194",  "BSNL")
    ],
    "Bengaluru": [
        ("49.204",   "Jio"),
        ("106.51",   "ACT"),
        ("117.192",  "BSNL"),
        ("125.16",   "Airtel")
    ],
    "Hyderabad": [
        ("49.205",   "Jio"),
        ("117.200",  "BSNL"),
        ("103.224",  "Airtel"),
        ("202.133",  "ACT")
    ],
    "Ahmedabad": [
        ("49.32",    "Jio"),
        ("117.211",  "BSNL"),
        ("103.48",   "Airtel"),
        ("59.185",   "GTPL")
    ],
    "Chennai": [
        ("49.207",   "Jio"),
        ("117.193",  "BSNL"),
        ("106.51",   "ACT"),
        ("103.21",   "Airtel")
    ],
    "Kolkata": [
        ("49.37",    "Jio"),
        ("103.24",   "Airtel"),
        ("117.194",  "BSNL"),
        ("59.93",    "Alliance")
    ],
    "Pune": [
        ("49.35",    "Jio"),
        ("103.48",   "Airtel"),
        ("117.208",  "BSNL"),
        ("59.88",    "Hathway")
    ],
    "Jaipur": [
        ("49.34",    "Jio"),
        ("117.219",  "BSNL"),
        ("103.22",   "Airtel"),
        ("59.95",    "Hathway")
    ],
    "Lucknow": [
        ("49.33",    "Jio"),
        ("117.223",  "BSNL"),
        ("103.23",   "Airtel"),
        ("59.97",    "Hathway")
    ],
    "Kanpur": [
        ("49.39",    "Jio"),
        ("117.225",  "BSNL"),
        ("103.20",   "Airtel"),
        ("59.98",    "Hathway")
    ],
    "Nagpur": [
        ("49.38",    "Jio"),
        ("117.227",  "BSNL"),
        ("103.19",   "Airtel"),
        ("59.92",    "Hathway")
    ],
    "Indore": [
        ("49.40",    "Jio"),
        ("117.229",  "BSNL"),
        ("103.25",   "Airtel"),
        ("59.91",    "Hathway")
    ],
    "Bhopal": [
        ("49.41",    "Jio"),
        ("117.231",  "BSNL"),
        ("103.26",   "Airtel"),
        ("59.94",    "Hathway")
    ],
    "Patna": [
        ("49.42",    "Jio"),
        ("117.233",  "BSNL"),
        ("103.27",   "Airtel"),
        ("59.96",    "Hathway")
    ],
    "Vadodara": [
        ("49.43",    "Jio"),
        ("117.235",  "BSNL"),
        ("103.28",   "Airtel"),
        ("59.99",    "GTPL")
    ],
    "Ludhiana": [
        ("49.44",    "Jio"),
        ("117.237",  "BSNL"),
        ("103.29",   "Airtel"),
        ("59.100",   "Hathway")
    ],
    "Agra": [
        ("49.45",    "Jio"),
        ("117.239",  "BSNL"),
        ("103.30",   "Airtel"),
        ("59.101",   "Hathway")
    ],
    "Varanasi": [
        ("49.46",    "Jio"),
        ("117.241",  "BSNL"),
        ("103.31",   "Airtel"),
        ("59.102",   "Hathway")
    ],
    "Surat": [
        ("49.47",    "Jio"),
        ("117.243",  "BSNL"),
        ("103.32",   "Airtel"),
        ("59.103",   "GTPL")
    ],
    "Coimbatore": [
        ("49.48",    "Jio"),
        ("117.245",  "BSNL"),
        ("103.33",   "Airtel"),
        ("59.104",   "Hathway")
    ],
    "Thane": [
        ("49.49",    "Jio"),
        ("117.247",  "BSNL"),
        ("103.34",   "Airtel"),
        ("59.105",   "Hathway")
    ],
    "Meerut": [
        ("49.50",    "Jio"),
        ("117.249",  "BSNL"),
        ("103.35",   "Airtel"),
        ("59.106",   "Hathway")
    ],
    "Rajkot": [
        ("49.51",    "Jio"),
        ("117.251",  "BSNL"),
        ("103.36",   "Airtel"),
        ("59.107",   "GTPL")
    ],
    "Ranchi": [
        ("49.52",    "Jio"),
        ("117.253",  "BSNL"),
        ("103.37",   "Airtel"),
        ("59.108",   "Hathway")
    ],
    "Guwahati": [
        ("49.53",    "Jio"),
        ("117.255",  "BSNL"),
        ("103.38",   "Airtel"),
        ("59.109",   "Hathway")
    ],
    "Chandigarh": [
        ("49.54",    "Jio"),
        ("117.257",  "BSNL"),
        ("103.39",   "Airtel"),
        ("59.110",   "Hathway")
    ],
    "Mysuru": [
        ("49.55",    "Jio"),
        ("117.259",  "BSNL"),
        ("103.40",   "Airtel"),
        ("59.111",   "ACT")
    ],
    "Dehradun": [
        ("49.56",    "Jio"),
        ("117.261",  "BSNL"),
        ("103.41",   "Airtel"),
        ("59.112",   "Hathway")
    ],
    "Visakhapatnam": [
        ("49.57",    "Jio"),
        ("117.263",  "BSNL"),
        ("103.42",   "Airtel"),
        ("59.113",   "ACT")
    ]
}


## **Helpers to build a safe IP and to pick based on city**

In [13]:
# --- Step 2: Function to generate random IP ---
def generate_ip(prefix):
    """Generate a random IPv4 address starting with given prefix (2 octets)."""
    return f"{prefix}.{random.randint(0,255)}.{random.randint(0,255)}"

# --- Step 3: Pick ISPs for a city ---
def get_city_isps(city, num_isps=2):
    """
    Select a city, pick num_isps ISPs randomly,
    and generate random IPs for them.
    """
    if city not in CITY_TO_ISP_PREFIXES:
        raise ValueError(f"{city} not in CITY_TO_ISP_PREFIXES")

    isp_prefixes = CITY_TO_ISP_PREFIXES[city]
    chosen = random.sample(isp_prefixes, k=min(num_isps, len(isp_prefixes)))

    results = []
    for prefix, isp in chosen:
        ip = generate_ip(prefix)
        results.append({"city": city, "isp": isp, "ip": ip})

    return results

# --- Example usage ---
city = "Mumbai"
profiles = get_city_isps(city, num_isps=3)
for p in profiles:
    print(p)

{'city': 'Mumbai', 'isp': 'ACT', 'ip': '106.51.55.217'}
{'city': 'Mumbai', 'isp': 'Tata', 'ip': '59.89.231.100'}
{'city': 'Mumbai', 'isp': 'Airtel', 'ip': '103.23.6.129'}


## **User generation logic**

In [22]:
# Define realistic device types distribution (India skewed heavily to Android)
device_types = ["Android"] * 7 + ["iOS"] * 3   # ~70% Android, ~30% iOS

users = []
user_ids = []

for i in range(NUM_USERS):
    # UPI ID
    upi_id = f"user{i}_{random.randint(1000,9999)}{random.choice(upi_id_domains)}"

    # Device
    device = random.choice(device_types)

    # City + lat/lon
    city_info = random.choices(indian_cities, weights=city_weights, k=1)[0]
    city_name = city_info["city"]
    lat, lon = city_info["lat"], city_info["lon"]

    # ISP + IP
    isp_prefixes = CITY_TO_ISP_PREFIXES[city_name]
    chosen_isps = random.sample(isp_prefixes, k=random.randint(2,3))  # pick 2–3 ISPs
    isp_details = [{"isp": isp, "ip": generate_ip(prefix)} for prefix, isp in chosen_isps]

    # Bank accounts (1–4 per user)
    bank_accounts = random.sample(banks, k=random.randint(1,4))

    profile = {
        "user_upi_id": upi_id,
        "device": device,
        "city": city_name,
        "latitude": lat,
        "longitude": lon,
        "isps": isp_details,
        "banks": bank_accounts
    }

    users.append(profile)
    user_ids.append(upi_id)

# Map UPI ID → user profile
user_map = {u['user_upi_id']: u for u in users}


# Map UPI ID → user profile
user_map = {u['user_upi_id']: u for u in users}

# Receiver pool: merchants + other users
receiver_ids_pool = [
    f"rec{i}_{random.randint(1000,9999)}{random.choice(upi_id_domains)}"
    for i in range(NUM_USERS * 2)
]
receiver_ids_pool += user_ids

In [23]:
print(f"Total users generated: {len(users)}")
print(f"Sample user IDs: {user_ids[:5]}")
print(f"Sample receiver IDs: {receiver_ids_pool[:5]}")
print(f"Sample user profiles:")
for u in users[:3]:
    print(u)


Total users generated: 5000
Sample user IDs: ['user0_8569@okaxis', 'user1_6188@sbi', 'user2_7034@apl', 'user3_2109@okaxis', 'user4_2862@sbi']
Sample receiver IDs: ['rec0_8755@hdfcbank', 'rec1_4937@okaxis', 'rec2_1860@hdfcbank', 'rec3_5104@okaxis', 'rec4_3578@apl']
Sample user profiles:
{'user_upi_id': 'user0_8569@okaxis', 'device': 'iOS', 'city': 'Surat', 'latitude': 21.1702, 'longitude': 72.8311, 'isps': [{'isp': 'GTPL', 'ip': '59.103.174.123'}, {'isp': 'Jio', 'ip': '49.47.89.231'}, {'isp': 'Airtel', 'ip': '103.32.22.95'}], 'banks': ['Indian Bank', 'IndusInd Bank', 'Federal Bank']}
{'user_upi_id': 'user1_6188@sbi', 'device': 'Android', 'city': 'Mumbai', 'latitude': 19.076, 'longitude': 72.8777, 'isps': [{'isp': 'ACT', 'ip': '106.51.82.222'}, {'isp': 'Jio', 'ip': '49.36.173.115'}, {'isp': 'Tata', 'ip': '59.89.190.54'}], 'banks': ['Union Bank of India', 'Canara Bank', 'Bank of India', 'Karur Vysya Bank']}
{'user_upi_id': 'user2_7034@apl', 'device': 'Android', 'city': 'Delhi', 'latitude'

## **Defining Time Span of Simulated Transactions**

In [24]:
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 3, 30)
avg_time_per_txn = (end_date - start_date).total_seconds() / TOTAL_TRANSACTIONS

## **Creating Possible Scenarios**

In [25]:
fraud_scenarios = [
    {'type': 'ATO', 'weight': 0.3},
    {'type': 'Phishing', 'weight': 0.2},
    {'type': 'Small-Large', 'weight': 0.2},
    {'type': 'High_Velocity', 'weight': 0.15},
    {'type': 'International_Anomaly', 'weight': 0.1},
    {'type': 'Failed_Attempts_Burst', 'weight': 0.05}
]

scenario_types = [s['type'] for s in fraud_scenarios]
scenario_weights = [s['weight'] for s in fraud_scenarios]


## **foreign Cities for International Anamoly**

In [27]:
# --- New Foreign City Data ---
foreign_cities = [
    {"city": "New York", "lat": 40.7128, "lon": -74.0060},
    {"city": "London", "lat": 51.5074, "lon": -0.1278},
    {"city": "Dubai", "lat": 25.2769, "lon": 55.2963},
    {"city": "Singapore", "lat": 1.3521, "lon": 103.8198},
    {"city": "Sydney", "lat": -33.8688, "lon": 151.2093}
]

## Foreign ISP

In [28]:
# --- New Foreign City-to-ISP Mapping ---
FOREIGN_CITY_TO_ISP = {
    "New York": [
        ("AT&T", 5, ["12.248", "68.232"]),
        ("Verizon", 4, ["96.224", "100.41"]),
        ("T-Mobile", 3, ["66.94", "172.58"])
    ],
    "London": [
        ("BT Group", 5, ["109.152", "62.64"]),
        ("Virgin Media", 3, ["82.35", "92.17"]),
        ("TalkTalk", 2, ["80.254", "88.96"])
    ],
    "Dubai": [
        ("Etisalat", 5, ["46.10", "94.200"]),
        ("Du", 3, ["95.138", "185.122"])
    ],
    "Singapore": [
        ("Singtel", 5, ["116.14", "202.156"]),
        ("StarHub", 3, ["203.116", "210.186"])
    ],
    "Sydney": [
        ("Telstra", 5, ["120.144", "203.206"]),
        ("Optus", 3, ["119.224", "1.144"]),
        ("Vodafone AU", 2, ["121.214", "14.201"])
    ]
}

In [30]:
transactions = []
current_timestamp = start_date

for _ in range(TOTAL_TRANSACTIONS):
    # Time increment
    current_timestamp += timedelta(seconds=int(avg_time_per_txn * random.uniform(0.5, 1.5)))
    if current_timestamp > end_date:
        current_timestamp = start_date + timedelta(
            seconds=random.randint(0, int((end_date - start_date).total_seconds()))
        )

    # Pick sender
    chosen_user = random.choice(list(user_map.keys()))
    sender = user_map[chosen_user]

    user_city = sender['city']
    lat, lon = sender['latitude'], sender['longitude']
    device_type = sender['device']
    sender_bank = random.choice(sender['banks'])

    # Pick ISP + IP
    isp_choice = random.choice(sender['isps'])
    isp, ip = isp_choice['isp'], isp_choice['ip']

    # Normal transaction setup
    transaction_type = random.choices(
        ['P2P', 'P2M', 'Bill Payment', 'Recharge'],
        weights=[0.6, 0.2, 0.1, 0.1]
    )[0]

    amount = round(random.uniform(10, 5000), 2)
    status = random.choices(['SUCCESS', 'FAILED'], weights=[0.98, 0.02])[0]
    receiver_bank = random.choice(banks)
    receiver_upi_id = f"user{random.randint(1000,9999)}@okbank"

    merchant_id, merchant_category = np.nan, np.nan
    is_fraud = 1 if random.random() < FRAUD_RATIO else 0

    # 🚨 Apply fraud scenarios
    if is_fraud:
        scenario = random.choices(scenario_types, weights=scenario_weights, k=1)[0]
        receiver_upi_id = f"fraud_rec_{fake.uuid4().split('-')[0]}@fraud"

        if scenario == 'ATO':
            device_type = random.choice([d['device'] for d in user_map.values() if d['device'] != sender['device']])
            isp, ip = "ForeignISP", fake.ipv4()
            user_city = random.choice(foreign_cities)
            lat, lon = random.uniform(0, 90), random.uniform(-180, 180)
            amount = round(random.uniform(50000, 300000), 2)

        elif scenario == 'Phishing':
            amount = round(random.uniform(10000, 200000), 2)
            if random.random() < 0.2:
                device_type = random.choice([d['device'] for d in user_map.values()])
            if random.random() < 0.2:
                user_city = random.choice(list({u['city'] for u in user_map.values()}))

        elif scenario == 'Small-Large':
            small_amt = round(random.uniform(1, 100), 2)
            transactions.append({
                'transaction_id': fake.uuid4(),
                'timestamp': current_timestamp - timedelta(minutes=random.randint(1, 5)),
                'user_upi_id': chosen_user,
                'receiver_upi_id': receiver_upi_id,
                'amount': small_amt,
                'transaction_type': 'P2P',
                'status': 'SUCCESS',
                'sender_bank': sender_bank,
                'receiver_bank': receiver_bank,
                'device_type': device_type,
                'ip_address': ip,
                'isp': isp,
                'city': user_city,
                'latitude': lat,
                'longitude': lon,
                'merchant_id': np.nan,
                'merchant_category': np.nan,
                'is_fraud': 1
            })
            amount = round(random.uniform(50000, 500000), 2)

        elif scenario == 'High_Velocity':
            burst_count = random.randint(3, 6)
            for _ in range(burst_count):
                transactions.append({
                    'transaction_id': fake.uuid4(),
                    'timestamp': current_timestamp + timedelta(seconds=random.randint(1, 60)),
                    'user_upi_id': chosen_user,
                    'receiver_upi_id': f"burst_rec_{fake.uuid4().split('-')[0]}@fraud",
                    'amount': round(random.uniform(50, 5000), 2),
                    'transaction_type': 'P2P',
                    'status': 'SUCCESS',
                    'sender_bank': sender_bank,
                    'receiver_bank': random.choice(banks),
                    'device_type': device_type,
                    'ip_address': ip,
                    'isp': isp,
                    'city': user_city,
                    'latitude': lat,
                    'longitude': lon,
                    'merchant_id': np.nan,
                    'merchant_category': np.nan,
                    'is_fraud': 1
                })
            amount = round(random.uniform(100, 5000), 2)

        elif scenario == 'International_Anomaly':
                    # Select a foreign city from the predefined list
                    foreign_city_info = random.choice(foreign_cities)
                    user_city = foreign_city_info["city"]
                    lat, lon = foreign_city_info["lat"], foreign_city_info["lon"]

                    # Select an ISP and prefix specific to the chosen foreign city
                    isp_list = FOREIGN_CITY_TO_ISP[user_city]
                    isp_names = [i[0] for i in isp_list]
                    isp_weights = [i[1] for i in isp_list]
                    isp_prefixes = {i[0]: i[2] for i in isp_list}

                    chosen_isp_name = random.choices(isp_names, weights=isp_weights, k=1)[0]
                    prefix = random.choice(isp_prefixes[chosen_isp_name])

                    # Update ISP and IP address
                    isp = chosen_isp_name
                    ip = f"{prefix}.{random.randint(0,255)}.{random.randint(0,255)}"

                    amount = round(random.uniform(20000, 1000000), 2)

        elif scenario == 'Failed_Attempts_Burst':
            for _ in range(random.randint(3, 7)):
                transactions.append({
                    'transaction_id': fake.uuid4(),
                    'timestamp': current_timestamp - timedelta(seconds=random.randint(10, 60)),
                    'user_upi_id': chosen_user,
                    'receiver_upi_id': receiver_upi_id,
                    'amount': round(random.uniform(100, 50000), 2),
                    'transaction_type': 'P2P',
                    'status': 'FAILED',
                    'sender_bank': sender_bank,
                    'receiver_bank': receiver_bank,
                    'device_type': device_type,
                    'ip_address': ip,
                    'isp': isp,
                    'city': user_city,
                    'latitude': lat,
                    'longitude': lon,
                    'merchant_id': np.nan,
                    'merchant_category': np.nan,
                    'is_fraud': 1
                })
            status = 'SUCCESS' if random.random() < 0.7 else 'FAILED'
            amount = round(random.uniform(1000, 100000), 2)

    # Re-check merchant logic
    if transaction_type == 'P2M':
        merchant_id = fake.uuid4()
        merchant_category = random.choice(merchant_categories)

    # Append final main transaction
    transactions.append({
        'transaction_id': fake.uuid4(),
        'timestamp': current_timestamp,
        'user_upi_id': chosen_user,
        'receiver_upi_id': receiver_upi_id,
        'amount': amount,
        'transaction_type': transaction_type,
        'status': status,
        'sender_bank': sender_bank,
        'receiver_bank': receiver_bank,
        'device_type': device_type,
        'ip_address': ip,
        'isp': isp,
        'city': user_city,
        'latitude': lat,
        'longitude': lon,
        'merchant_id': merchant_id,
        'merchant_category': merchant_category,
        'is_fraud': is_fraud
    })


In [31]:
df = pd.DataFrame(transactions)
print(f"✅ Generated {len(df)} transactions.")
print(f"🟥 Fraud cases (random only): {df['is_fraud'].sum()} ({df['is_fraud'].mean()*100:.2f}%)")
print(df.sample(5))


✅ Generated 151629 transactions.
🟥 Fraud cases (random only): 3120 (2.06%)
                              transaction_id           timestamp  \
143873  b3f12c72-d214-47d9-932d-fb194bb1ff64 2025-03-24 18:18:15   
73785   456a8775-1ebc-4688-9e41-b6bb6cd96dbe 2025-02-12 10:12:45   
56067   5b8f81df-fc67-489b-82c9-4867c7224665 2025-02-02 05:31:35   
31520   80d09564-e6fe-4237-944b-5a147e7c6a4b 2025-01-19 03:04:38   
5222    4ca5faa7-d8eb-4359-8d2b-8e490d3bb39f 2025-01-03 23:26:20   

                 user_upi_id  receiver_upi_id   amount transaction_type  \
143873   user3445_8324@ptsbi  user9807@okbank  1683.40              P2M   
73785   user82_1766@hdfcbank  user4121@okbank  4689.47              P2P   
56067      user3903_6166@sbi  user3023@okbank  4576.13              P2P   
31520   user3314_1021@okaxis  user2782@okbank  4921.07              P2P   
5222        user760_3039@apl  user2448@okbank  4783.13              P2P   

         status              sender_bank          receiver_bank d

In [33]:
import pandas as pd

# Create DataFrame if not already done
df = pd.DataFrame(transactions)

# Sort by timestamp (optional, but useful)
df = df.sort_values(by='timestamp').reset_index(drop=True)

# Save to CSV
df.to_csv("upi-transaction-dataset-01.csv", index=False)

print("✅ CSV saved as 'upi-transaction-dataset-01.csv'")
print(f"Total transactions: {len(df)} | Fraud: {df['is_fraud'].sum()} ({df['is_fraud'].mean() * 100:.2f}%)")


✅ CSV saved as 'upi-transaction-dataset-01.csv'
Total transactions: 151629 | Fraud: 3120 (2.06%)


In [34]:
df.head(10)


,transaction_id,timestamp,user_upi_id,receiver_upi_id,amount,transaction_type,status,sender_bank,receiver_bank,device_type,ip_address,isp,city,latitude,longitude,merchant_id,merchant_category,is_fraud
0,067bcfe2-55b8-44b3-9d46-20762002f130,2025-01-01 00:01:01,user1950_3682@sbi,user6472@okbank,4655.11,P2P,SUCCESS,RBL Bank,Federal Bank,Android,117.194.12.168,BSNL,Delhi,28.7041,77.1025,NaN,NaN,0
1,b30c1893-10f6-4ac4-8ab0-52f6f40b3d82,2025-01-01 00:01:47,user1025_7340@hdfcbank,user9006@okbank,1822.92,Bill Payment,SUCCESS,IDFC FIRST Bank,City Union Bank,Android,117.194.68.201,BSNL,Delhi,28.7041,77.1025,NaN,NaN,0
2,8a0362a2-703a-4336-9d7e-7b4b39fe2118,2025-01-01 00:02:14,user532_3937@ptsbi,user7167@okbank,4408.68,Bill Payment,SUCCESS,AU Small Finance Bank,RBL Bank,Android,49.32.253.38,Jio,Ahmedabad,23.0225,72.5714,NaN,NaN,0
3,cdb99a42-9d34-451b-bae6-2a6dffc8bb9d,2025-01-01 00:03:20,user379_4614@sbi,user6353@okbank,2023.02,Recharge,SUCCESS,Indian Bank,AU Small Finance Bank,iOS,49.57.99.74,Jio,Visakhapatnam,17.6868,83.2185,NaN,NaN,0
4,3b35c751-71b5-4633-9550-ae267247b532,2025-01-01 00:04:26,user286_6095@apl,user1802@okbank,1156.38,Bill Payment,SUCCESS,Karur Vysya Bank,South Indian Bank,Android,59.91.159.33,Hathway,Indore,22.7196,75.8577,NaN,NaN,0
5,5c272fb7-3b57-4329-93fe-d5ec58c6c90b,2025-01-01 00:05:15,user2833_6036@sbi,user3481@okbank,1013.22,Recharge,SUCCESS,South Indian Bank,Canara Bank,iOS,106.51.130.241,ACT,Mumbai,19.0760,72.8777,NaN,NaN,0
6,4967b86e-56a6-4234-8c09-b92334d41133,2025-01-01 00:06:17,user4809_3433@ybl,user1380@okbank,634.28,Recharge,SUCCESS,SBI,Punjab National Bank,iOS,59.110.189.6,Hathway,Chandigarh,30.7333,76.7794,NaN,NaN,0
7,effeea95-2308-4331-95b8-61cc5a932758,2025-01-01 00:07:27,user2724_6599@okaxis,user3807@okbank,4361.26,P2P,SUCCESS,HDFC Bank,South Indian Bank,Android,49.205.183.2,Jio,Delhi,28.7041,77.1025,NaN,NaN,0
8,79072da3-d252-4217-839c-a85ad46cbdc6,2025-01-01 00:08:04,user132_1622@sbi,user4589@okbank,4193.42,P2P,SUCCESS,IDFC FIRST Bank,Canara Bank,Android,27.97.28.24,Airtel,Delhi,28.7041,77.1025,NaN,NaN,0
9,e8297a91-b090-4670-bd9a-efc9d03a71f8,2025-01-01 00:09:08,user1392_2002@apl,user9046@okbank,1613.36,P2P,SUCCESS,Central Bank of India,IDFC FIRST Bank,Android,59.92.54.136,Hathway,Nagpur,21.1458,79.0882,NaN,NaN,0


In [35]:
from tabulate import tabulate

In [37]:

target_upi_id = "user132_1622@sbi"

# Filter the rows
user_txns = df[df['user_upi_id'] == target_upi_id]

# Filter transactions for that user
user_txns = df[df['user_upi_id'] == target_upi_id]

# Display as pretty table
print(f"\n Showing {len(user_txns)} transactions for: {target_upi_id}\n")
print(tabulate(user_txns, headers='keys', tablefmt='grid', showindex=False))


 Showing 35 transactions for: user132_1622@sbi

+--------------------------------------+---------------------+------------------+--------------------------+-----------+--------------------+----------+---------------------+----------------------+---------------+-----------------+----------+--------+------------+-------------+--------------------------------------+---------------------+------------+
| transaction_id                       | timestamp           | user_upi_id      | receiver_upi_id          |    amount | transaction_type   | status   | sender_bank         | receiver_bank        | device_type   | ip_address      | isp      | city   |   latitude |   longitude | merchant_id                          | merchant_category   |   is_fraud |
+======================================+=====================+==================+==========================+===========+====================+==========+=====================+======================+===============+=================+==========+===

In [ ]:
from google.colab import files
files.download('upi_labeled_transactions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151629 entries, 0 to 151628
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   transaction_id     151629 non-null  object        
 1   timestamp          151629 non-null  datetime64[ns]
 2   user_upi_id        151629 non-null  object        
 3   receiver_upi_id    151629 non-null  object        
 4   amount             151629 non-null  float64       
 5   transaction_type   151629 non-null  object        
 6   status             151629 non-null  object        
 7   sender_bank        151629 non-null  object        
 8   receiver_bank      151629 non-null  object        
 9   device_type        151629 non-null  object        
 10  ip_address         151629 non-null  object        
 11  isp                151629 non-null  object        
 12  city               151629 non-null  object        
 13  latitude           151629 non-null  float64 